## Mix de produção

Uma Empresa deseja programar a produção de um utensílio de cozinha 
que requer o uso de dois tipos de recursos: mão-de-obra e material. Ela está 
considerando a fabricação de três modelos e o seu Departamento de Engenharia forneceu os dados a seguir (Tabela 1). O suprimento de material é de 
200 quilos por dia. A disponibilidade diária de mão-de–obra é 150 horas. 
Formule um modelo de programação linear para determinar a produção 
diária de cada um dos modelos de modo a maximizar o lucro total da Empresa. 

![Tabela 1](img/exe-1.png)

---

### Modelagem:

**Variáveis de decisão:**

- $ x_a $ – produção diária do modelo **a**  
- $ x_b $ – produção diária do modelo **b**  
- $ x_c $ – produção diária do modelo **c**


---

**Restrições:**

*(limitação de mão-de-obra)*  

$
7x_a + 3x_b + 6x_c \leq 150
$

*(limitação de material)*  

$
4x_a + 4x_b + 5x_c \leq 200
$

*(não-negatividade)*  

$
x_a \geq 0, \quad x_b \geq 0, \quad x_c \geq 0
$

---

**Função objetivo: maximização do lucro total**

Lucro total:  

$
L = 4x_a + 2x_b + 3x_c
$

---

**Modelo final:**  
Encontrar números 
$
 x_a, x_b, x_c 
$ tais que:

$
\max L = 4x_a + 2x_b + 3x_c
$

sujeito às restrições:

$
\begin{cases}
7x_a + 3x_b + 6x_c \leq 150 \\
4x_a + 4x_b + 5x_c \leq 200 \\
x_a \geq 0, \quad x_b \geq 0, \quad x_c \geq 0
\end{cases}
$

---

### Generalizando

Suponha que existam $m$ recursos usados na produção de $n$ produtos, com os seguintes dados:

- $c_j$: lucro na venda de uma unidade do produto $j = 1, 2, \dots, n$; 
- $b_i$: quantidade disponível do recurso $i = 1, 2, \dots, m$;
- $a_{ij}$: quantidade do recurso $i$ usada para produzir uma unidade do produto $j$.
- $x_j$: quantidade a produzir do produto $j$ (variável de decisão).

O modelo geral terá:

**Função objetivo – Maximizar**

$
Z = \sum_{j=1}^{n} c_j x_j
$

**Sujeito às restrições:**

$
\begin{cases}
a_{11}x_1 + a_{12}x_2 + \cdots + a_{1n}x_n \leq b_1 \\
a_{21}x_1 + a_{22}x_2 + \cdots + a_{2n}x_n \leq b_2 \\
\vdots \\
a_{m1}x_1 + a_{m2}x_2 + \cdots + a_{mn}x_n \leq b_m \\
x_j \geq 0, \quad j = 1, 2, \dots, n
\end{cases}
$

---

### Em notação matricial:

$
\max Z = C'X \quad \text{sujeito a:} \quad
\begin{cases}
AX \leq b \\
X \geq 0
\end{cases}
$

Onde:

$
X = \begin{pmatrix} x_1 & x_2 & \cdots & x_n \end{pmatrix}^t
$
é o vetor das variáveis de decisão,

$
C = \begin{pmatrix} c_1 & c_2 & \cdots & c_n \end{pmatrix}^t
$
é o vetor dos lucros,

$
b = \begin{pmatrix} b_1 & b_2 & \cdots & b_m \end{pmatrix}^t
$
é o vetor das quantidades dos recursos, e

$
A = 
\begin{pmatrix}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{pmatrix}
$
é a matriz dos coeficientes tecnológicos.


In [1]:
import pulp as p
import pandas as pd

# Dados de entrada
df = pd.DataFrame({
    'Modelo' :     ['a','b','c'],
    'mao_de_obra': [7,3,6],
    'material':    [4,4,5],
    'lucro' :      [4,2,3]
})

print(df)

modelo = p.LpProblem("Maximizar_Lucro", p.LpMaximize)

# Variáveis de decisão
decisao = {linha.Modelo: p.LpVariable(f"x_{linha.Modelo}", lowBound=0, cat='Integer')
	for linha in df.itertuples()
}

# Função objetivo
modelo += p.lpSum(decisao[i] * df.loc[df.Modelo == i, 'lucro'].iloc[0] for i in df.Modelo)

# Restrições
modelo += p.lpSum(decisao[i] * df.loc[df.Modelo == i, 'mao_de_obra'].iloc[0] for i in df.Modelo) <= 200
modelo += p.lpSum(decisao[i] * df.loc[df.Modelo == i, 'material'].iloc[0] for i in df.Modelo) <= 150

modelo.solve()

print("Status:", p.LpStatus[modelo.status])
for i in df.Modelo:
	print(f"Produzir {decisao[i].varValue} unidades do produto {i}")
print("Lucro total: R$", p.value(modelo.objective))

  Modelo  mao_de_obra  material  lucro
0      a            7         4      4
1      b            3         4      2
2      c            6         5      3
Status: Optimal
Produzir 22.0 unidades do produto a
Produzir 15.0 unidades do produto b
Produzir 0.0 unidades do produto c
Lucro total: R$ 118.0
